#### Train VITS model using restore path

    In this notebook I restore from a model and start training from that checkpoint

##### Setup

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

##### Dataset

In [ ]:
output_path = "TTS_train"
#print(os.path.join(output_path, "LJSpeech-1.1/"))
dataset_config = BaseDatasetConfig(
    name="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "LJSpeech-1.1/")
)


##### Config

In [ ]:
# Removed a few config, because the VITs model has roughly 83 million parameters
# batch_group_size=5
# num_loader_workers=8, -- changed to 4
# eval_batch_size=16,

config = VitsConfig(
    run_name="vits_ljspeech_finetune",
    batch_size=64,
    eval_batch_size=8,
    num_loader_workers=8,
    num_eval_loader_workers=4,
    run_eval=False,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    max_text_len=200,
    datasets=[dataset_config],
    cudnn_benchmark=False,
    save_step=1000,
    batch_group_size=2,
    lr_gen=0.0001,
    lr_disc=0.0001,
)

In [ ]:
# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

In [ ]:
# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

##### Train Eval split

In [ ]:
# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

##### Train: Get set go

In [ ]:
CONFIG_PATH="./VITS_MODEL_LJSPEECH/config.json"
config.save_json(CONFIG_PATH)

In [ ]:

MODEL_PATH = "./VITS_MODEL_LJSPEECH/model_file.pth"
command = f"python ./TTS/TTS/bin/train_tts.py --config_path {CONFIG_PATH} --restore_path {MODEL_PATH}"
os.system(command)

##### Tensorboard 

In [ ]:
!pip install tensorboard
!tensorboard --logdir=TTS_train

In [ ]:
# USe this cell to see if cuda memory does not remain stuck
import torch
print(torch.cuda.memory_reserved())
print(torch.cuda.memory_allocated())
torch.cuda.empty_cache()
print(torch.cuda.memory_reserved())
print(torch.cuda.memory_allocated())

In [ ]:
torch.cuda.is_available()
print(torch.__version__)

##### Test the model

In [ ]:
import glob, os
output_path = "TTS_train"
ckpt = output_path + "best_model.pth"
config = output_path + "config.json"

In [ ]:
!tts --text "Is it recognize speech or wreck a nice beach?" \
      --model_path $ckpt \
      --config_path $config \
      --out_path test1.wav

##### Listen to the synthesized speech

In [ ]:
import IPython
IPython.display.Audio("test1.wav")